<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

# Photutils Overview Exercise Solutions

## Exercise 1

The [**data/**](data) subdirectory also contains a WFC3/IR F105W image of the same field used for the preceding examples. The F105W and F160W images are pixel aligned, so sources in the F105W image are located at the same detected positions as the F160W image (if they are visible in the F105W image). 

In the F105W image, find the count rate in a circular aperture with a 5-pixel radius of the 3 brightest sources in isophotal flux (i.e., the `segment_flux` column) detected in the F160W image.

Hint: we previously saved the F160W catalog to an ECSV file called `xdf_f160w_catalog.ecsv`

In [ ]:
from astropy.io import fits
from astropy.table import QTable
import astropy.units as u
from photutils.aperture import CircularAperture, aperture_photometry
from photutils.segmentation import SegmentationImage

In [ ]:
# read in the F105W science data 
f105w_data = fits.getdata('data/xdf_hst_wfc3ir_60mas_f105w_sci.fits')
f105w_data *= (u.electron / u.s)

In [ ]:
import os

# these files were saved in the photutils_overview.ipynb notebook
f160w_catalog_filename = 'xdf_f160w_catalog.ecsv'
f160w_segm_filename = 'xdf_f160w_segm.fits'

# This check is needed for continuous integration (CI) on GitHub.
has_files = (os.path.exists(f160w_catalog_filename) &
             os.path.exists(f160w_segm_filename))

# read in the table of sources detected in the F160W image
# and the F160W segmentation image
if has_files:
    f160w_tbl = QTable.read(f160w_catalog_filename)
    
    f160w_segm_data = fits.getdata(f160w_segm_filename)
    f160w_segm = SegmentationImage(f160w_segm_data)

In [ ]:
if has_files:
    # sort table by isophotal flux (the segment_flux column) and select the 3 brightest
    idx = np.argsort(f160w_tbl['segment_flux'])[::-1][:3]
    brightest = f160w_tbl[idx]
    print(brightest)

In [ ]:
if has_files:
    # use the (x, y) coordinates from the F160W table
    xypos = np.transpose((brightest['xcentroid'], brightest['ycentroid']))
    aperture = CircularAperture(xypos, r=5)
    f105w_phot = aperture_photometry(f105w_data, aperture)
    print(f105w_phot)

## Exercise 2

Calculate the $Y_{105} - H_{160}$ (F105W $-$ F160W) isophotal colors for all sources detected in the F160W image.

The WFC3/IR F105W and F160W AB magnitude zero points are 26.2687 and 25.9463, respectively.

Hints: 

* The isophotal fluxes are found in the `segment_flux` column.

* Because the images are pixel aligned, the F160W segmentation image can be directly applied to the F105W image.

* We previously saved the F160W segmentation image in a file called `xdf_f160w_segm.fits`.

In [ ]:
from photutils.segmentation import SourceCatalog

if has_files:
    f105w_cat = SourceCatalog(f105w_data, f160w_segm)

In [ ]:
if has_files:
    f105w_abmag = -2.5 * np.log10(f105w_cat.segment_flux.value) + 26.2687
    f160w_abmag = -2.5 * np.log10(f160w_tbl['segment_flux'].value) + 26.9463
    
    yh_color = f105w_abmag - f160w_abmag
    print(yh_color)